# Load the data

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

# Load data from the SQLite database
def load_data_from_db(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Fetch all state and solution length pairs
    cursor.execute("SELECT state, solution_length FROM puzzle_states")
    data = cursor.fetchall()

    # Close the connection
    conn.close()

    # Create a DataFrame from the fetched data
    df = pd.DataFrame(data, columns=['state', 'solution_length'])

    return df

# Calculate the Manhattan distance for each puzzle state
def calculate_manhattan_distance(state, goal_state):
    total_distance = 0
    state_reshaped = np.reshape(state, (3, 3))  # Convert the flat state into a 3x3 grid
    for i in range(3):
        for j in range(3):
            tile = state_reshaped[i][j]
            if tile != 0:  # Ignore the blank tile
                goal_i, goal_j = [(ix, iy) for ix, row in enumerate(goal_state) for iy, val in enumerate(row) if val == tile][0]
                total_distance += abs(i - goal_i) + abs(j - goal_j)
    return total_distance

# Preprocess the data: load the state, compute Manhattan distance, and return a DataFrame
def preprocess_data(df):
    goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]  # Define the goal state
    manhattan_distances = []

    # Compute Manhattan distance for each state
    for state_str in df['state']:
        state_list = eval(state_str)  # Convert the string representation of the state to a list
        flattened_state = [tile for row in state_list for tile in row]  # Flatten the state
        manhattan_distance = calculate_manhattan_distance(flattened_state, goal_state)
        manhattan_distances.append(manhattan_distance)

    # Add the Manhattan distance to the DataFrame
    df['manhattan_distance'] = manhattan_distances

    return df

# Specify the path to your SQLite database
db_path = '/content/puzzle_database.db'

# Load the data from the database
df = load_data_from_db(db_path)

# Preprocess the data (compute Manhattan distance)
df = preprocess_data(df)

# Check if the data looks correct
print(df.head())


                               state  solution_length  manhattan_distance
0  [[1, 2, 3], [4, 5, 6], [7, 8, 0]]                0                   0
1  [[1, 2, 3], [4, 5, 6], [7, 0, 8]]                1                   1
2  [[1, 2, 3], [4, 5, 0], [7, 8, 6]]                1                   1
3  [[1, 2, 3], [4, 5, 6], [0, 7, 8]]                2                   2
4  [[1, 2, 3], [4, 0, 6], [7, 5, 8]]                2                   2


# Train a Simple Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Extract the Manhattan distance as the feature and solution length as the target
X = df['manhattan_distance'].values.reshape(-1, 1)  # Feature: Manhattan distance
y = df['solution_length'].values  # Target: Solution length

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Evaluate the model
y_pred = lr_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Linear Regression - MSE: {mse}, MAE: {mae}")


Linear Regression - MSE: 7.779769029707702, MAE: 2.1957904558767747


# PuzzleSolver

In [ ]:
import heapq
import time
import numpy as np

class Node:
    def __init__(self, state, level, f_value, parent=None):
        self.state = state
        self.level = level  # Number of moves taken to reach this state
        self.f_value = f_value  # f(n) = g(n) + h(n), g(n) is the level, h(n) is the heuristic
        self.parent = parent  # Pointer to the parent node

    def __lt__(self, other):
        return self.f_value < other.f_value  # For priority queue comparison


class PuzzleSolver:
    def __init__(self, model=None):
        self.model = model  # The machine learning model for the learned heuristic

    def heuristic(self, state):
        """Compute the heuristic based on the model or Manhattan distance."""
        if self.model:
            # Flatten the state to pass it into the model
            flattened_state = [tile for row in state for tile in row]

            # Assume Manhattan distance as a feature (optional)
            goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
            manhattan_distance = calculate_manhattan_distance(flattened_state, goal_state)

            # Use the model to predict the heuristic value
            return self.model.predict([[manhattan_distance]])[0]  # Use the model's prediction
        else:
            # Traditional Manhattan distance heuristic
            goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
            return self.manhattan_distance(state, goal_state)

    def manhattan_distance(self, state, goal_state):
        """Calculate the Manhattan distance heuristic."""
        total_distance = 0
        for i in range(3):
            for j in range(3):
                tile = state[i][j]
                if tile != 0:
                    goal_i, goal_j = [(ix, iy) for ix, row in enumerate(goal_state) for iy, val in enumerate(row) if val == tile][0]
                    total_distance += abs(i - goal_i) + abs(j - goal_j)
        return total_distance

    def generate_children(self, node):
        """Generate child states by moving the blank tile (0)."""
        children = []
        n = 3
        moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # right, left, down, up
        x, y = [(ix, iy) for ix, row in enumerate(node.state) for iy, val in enumerate(row) if val == 0][0]  # Find 0

        for move in moves:
            new_x, new_y = x + move[0], y + move[1]
            if 0 <= new_x < n and 0 <= new_y < n:
                new_state = [row[:] for row in node.state]  # Create a copy of the state
                new_state[x][y], new_state[new_x][new_y] = new_state[new_x][new_y], new_state[x][y]
                child_node = Node(new_state, node.level + 1, 0, node)
                children.append(child_node)

        return children

    def solve(self, initial_state, goal_state):
        """Implement the A* search algorithm with metrics collection."""
        open_list = []
        closed_set = set()

        initial_node = Node(initial_state, 0, 0, None)
        initial_node.f_value = self.heuristic(initial_state)
        heapq.heappush(open_list, initial_node)

        solution_length = 0
        nodes_expanded = 0  # Initialize counter

        start_time = time.time()  # Start timing the search

        while open_list:
            current_node = heapq.heappop(open_list)
            closed_set.add(tuple(map(tuple, current_node.state)))
            nodes_expanded += 1  # Increment nodes expanded

            if current_node.state == goal_state:
                path = []
                while current_node:
                    path.append(current_node.state)
                    current_node = current_node.parent
                    solution_length += 1

                end_time = time.time()  # End timing the search
                search_time = end_time - start_time  # Calculate the time taken

                return (list(reversed(path)), solution_length, nodes_expanded, search_time)

            # Generate children nodes
            children = self.generate_children(current_node)
            for child in children:
                if tuple(map(tuple, child.state)) not in closed_set:
                    child.f_value = child.level + self.heuristic(child.state)
                    heapq.heappush(open_list, child)

        end_time = time.time()  # End timing the search
        search_time = end_time - start_time

        return (None, solution_length, nodes_expanded, search_time)


# Using the PuzzleSolver Class with the Linear Regression Model:

In [ ]:
# Instantiate the PuzzleSolver class with the Linear Regression model
solver_learned = PuzzleSolver(model=lr_model)

# Define the initial state and goal state
initial_state = [[7, 2, 4], [5, 0, 6], [8, 3, 1]]  # Example initial state
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]  # Goal state

# Solve the puzzle using the learned heuristic (Linear Regression model)
result_learned = solver_learned.solve(initial_state, goal_state)

# Print the results for the learned heuristic
print("Learned Heuristic Results:")
print("Solution Length:", result_learned[1])
print("Nodes Expanded:", result_learned[2])
print("Search Time:", result_learned[3])

# Instantiate the PuzzleSolver class for the traditional heuristic (Manhattan distance)
solver_traditional = PuzzleSolver()

# Solve the puzzle using the traditional heuristic (Manhattan distance)
result_traditional = solver_traditional.solve(initial_state, goal_state)

# Print the results for the traditional heuristic
print("\nTraditional Heuristic Results:")
print("Solution Length:", result_traditional[1])
print("Nodes Expanded:", result_traditional[2])
print("Search Time:", result_traditional[3])


Learned Heuristic Results:
Solution Length: 21
Nodes Expanded: 1525
Search Time: 0.4467661380767822

Traditional Heuristic Results:
Solution Length: 21
Nodes Expanded: 273
Search Time: 0.013576269149780273


# Trying with Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Use the Random Forest model as the heuristic
solver_learned_rf = PuzzleSolver(model=rf_model)

# Solve the puzzle using the Random Forest model
result_learned_rf = solver_learned_rf.solve(initial_state, goal_state)

# Print the results for the learned heuristic (Random Forest)
print("Learned Heuristic (Random Forest) Results:")
print("Solution Length:", result_learned_rf[1])
print("Nodes Expanded:", result_learned_rf[2])
print("Search Time:", result_learned_rf[3])


Learned Heuristic (Random Forest) Results:
Solution Length: 21
Nodes Expanded: 334
Search Time: 2.8943030834198


# Let's add more Features

# Load the data and compute Manhattan distance

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

# Load data from the SQLite database
def load_data_from_db(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT state, solution_length FROM puzzle_states")
    data = cursor.fetchall()
    conn.close()
    df = pd.DataFrame(data, columns=['state', 'solution_length'])
    return df

# Calculate the Manhattan distance
def calculate_manhattan_distance(state, goal_state):
    total_distance = 0
    state_reshaped = np.reshape(state, (3, 3))
    for i in range(3):
        for j in range(3):
            tile = state_reshaped[i][j]
            if tile != 0:
                goal_i, goal_j = [(ix, iy) for ix, row in enumerate(goal_state) for iy, val in enumerate(row) if val == tile][0]
                total_distance += abs(i - goal_i) + abs(j - goal_j)
    return total_distance

def calculate_misplaced_tiles(state, goal_state):
    misplaced = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0 and state[i][j] != goal_state[i][j]:
                misplaced += 1
    return misplaced

def calculate_tile_position_indicators(state, goal_state):
    correct_row = correct_col = 0
    for i in range(3):
        for j in range(3):
            tile = state[i][j]
            if tile != 0:
                goal_i, goal_j = [(ix, iy) for ix, row in enumerate(goal_state) for iy, val in enumerate(row) if val == tile][0]
                if i == goal_i: correct_row += 1
                if j == goal_j: correct_col += 1
    return correct_row, correct_col

# Preprocess the data: load state and compute additional features
def preprocess_data(df):
    goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
    manhattan_distances = []
    misplaced_tiles = []
    correct_row_col = []

    for state_str in df['state']:
        state_list = eval(state_str)  # Convert the string to list
        flattened_state = [tile for row in state_list for tile in row]
        manhattan_distance = calculate_manhattan_distance(flattened_state, goal_state)
        misplaced = calculate_misplaced_tiles(state_list, goal_state)
        row_col = calculate_tile_position_indicators(state_list, goal_state)

        manhattan_distances.append(manhattan_distance)
        misplaced_tiles.append(misplaced)
        correct_row_col.append(row_col)

    df['manhattan_distance'] = manhattan_distances
    df['misplaced_tiles'] = misplaced_tiles
    df['correct_row'], df['correct_column'] = zip(*correct_row_col)

    return df

# Load the data
db_path = '/content/puzzle_database.db'  # Update with your path
df = load_data_from_db(db_path)

# Preprocess the data (calculate features)
df = preprocess_data(df)


# Train the Model with Additional Features

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Use the additional features in the model
X = df[['manhattan_distance', 'misplaced_tiles', 'correct_row', 'correct_column']].values  # Features
y = df['solution_length'].values  # Target (solution length)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the Random Forest model
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Random Forest - MSE: {mse}, MAE: {mae}")


Random Forest - MSE: 6.27446531719076, MAE: 1.9852507638124954


# Puzzle Solver

In [ ]:
import heapq
import numpy as np

class PuzzleSolver:
    def __init__(self, model=None):
        self.model = model  # The machine learning model for the learned heuristic

    def heuristic(self, state):
        """Compute the heuristic based on the model or Manhattan distance."""
        if self.model:
            # Flatten the state for feature calculation
            flattened_state = [tile for row in state for tile in row]

            # Compute all features needed for the model
            goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
            manhattan_distance = calculate_manhattan_distance(flattened_state, goal_state)
            misplaced_tiles = calculate_misplaced_tiles(state, goal_state)
            correct_row, correct_col = calculate_tile_position_indicators(state, goal_state)

            # Create the feature vector
            feature_vector = np.array([[manhattan_distance, misplaced_tiles, correct_row, correct_col]])

            # Use the model to predict the heuristic value
            return self.model.predict(feature_vector)[0]
        else:
            # Use the traditional Manhattan distance heuristic if no model is provided
            goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
            return self.manhattan_distance(state, goal_state)

    def manhattan_distance(self, state, goal_state):
        """Calculate the Manhattan distance heuristic."""
        total_distance = 0
        for i in range(3):
            for j in range(3):
                tile = state[i][j]
                if tile != 0:
                    goal_i, goal_j = [(ix, iy) for ix, row in enumerate(goal_state) for iy, val in enumerate(row) if val == tile][0]
                    total_distance += abs(i - goal_i) + abs(j - goal_j)
        return total_distance

    def generate_children(self, node):
        """Generate child states by moving the blank tile (0)."""
        children = []
        n = 3
        moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # right, left, down, up
        x, y = [(ix, iy) for ix, row in enumerate(node.state) for iy, val in enumerate(row) if val == 0][0]  # Find 0

        for move in moves:
            new_x, new_y = x + move[0], y + move[1]
            if 0 <= new_x < n and 0 <= new_y < n:
                new_state = [row[:] for row in node.state]  # Create a copy of the state
                new_state[x][y], new_state[new_x][new_y] = new_state[new_x][new_y], new_state[x][y]
                child_node = Node(new_state, node.level + 1, 0, node)
                children.append(child_node)

        return children

    def solve(self, initial_state, goal_state):
        """Implement the A* search algorithm with metrics collection."""
        open_list = []
        closed_set = set()

        initial_node = Node(initial_state, 0, 0, None)
        initial_node.f_value = self.heuristic(initial_state)
        heapq.heappush(open_list, initial_node)

        solution_length = 0
        nodes_expanded = 0  # Initialize counter

        start_time = time.time()  # Start timing the search

        while open_list:
            current_node = heapq.heappop(open_list)
            closed_set.add(tuple(map(tuple, current_node.state)))
            nodes_expanded += 1  # Increment nodes expanded

            if current_node.state == goal_state:
                path = []
                while current_node:
                    path.append(current_node.state)
                    current_node = current_node.parent
                    solution_length += 1

                end_time = time.time()  # End timing the search
                search_time = end_time - start_time  # Calculate the time taken

                return (list(reversed(path)), solution_length, nodes_expanded, search_time)

            # Generate children nodes
            children = self.generate_children(current_node)
            for child in children:
                if tuple(map(tuple, child.state)) not in closed_set:
                    child.f_value = child.level + self.heuristic(child.state)
                    heapq.heappush(open_list, child)

        end_time = time.time()  # End timing the search
        search_time = end_time - start_time

        return (None, solution_length, nodes_expanded, search_time)


# Use the Model in A* Search

In [ ]:
# Instantiate the PuzzleSolver class with the Random Forest model
solver_learned = PuzzleSolver(model=rf_model)

# Solve the puzzle using the learned heuristic (Random Forest model)
result_learned = solver_learned.solve(initial_state, goal_state)

# Print the results for the learned heuristic
print("Learned Heuristic Results:")
print("Solution Length:", result_learned[1])
print("Nodes Expanded:", result_learned[2])
print("Search Time:", result_learned[3])


Learned Heuristic Results:
Solution Length: 21
Nodes Expanded: 128
Search Time: 1.1088335514068604


# Tuning Random Forest with Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Random Forest
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Perform Grid Search to find the best parameters
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best model from the grid search
best_rf_model = grid_search.best_estimator_

# Use the best model to solve the puzzle
solver_learned = PuzzleSolver(model=best_rf_model)
result_learned = solver_learned.solve(initial_state, goal_state)

# Print results
print("Learned Heuristic Results (Best RF Model):")
print("Solution Length:", result_learned[1])
print("Nodes Expanded:", result_learned[2])
print("Search Time:", result_learned[3])


Learned Heuristic Results (Best RF Model):
Solution Length: 21
Nodes Expanded: 131
Search Time: 2.6688272953033447


# XGBoost with Additional Features

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Define parameter grid for XGBoost
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.3]
}

# Perform grid search
xgb = XGBRegressor()
grid_search = GridSearchCV(xgb, param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best model
best_xgb_model = grid_search.best_estimator_

# Use the best model to solve the puzzle
solver_learned = PuzzleSolver(model=best_xgb_model)
result_learned = solver_learned.solve(initial_state, goal_state)

# Print results
print("Learned Heuristic Results (Best XGBoost Model):")
print("Solution Length:", result_learned[1])
print("Nodes Expanded:", result_learned[2])
print("Search Time:", result_learned[3])


Learned Heuristic Results (Best XGBoost Model):
Solution Length: 21
Nodes Expanded: 127
Search Time: 0.15287423133850098
